In [1]:
import pandas as pd
import numpy as np
import clickhouse_connect
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import time
from datetime import datetime
from dateutil.tz import tzutc, tzlocal

In [2]:
clickhouse_client = clickhouse_connect.get_client(
    host="clickhouse.matter.bytebeam.io",
    port=443,
    username="matter_rw",
    password="risazAza9saNGEb8",
    secure=True,
    query_limit=0)

In [6]:
utc = datetime.now(tzutc())
index = clickhouse_client.query(f"SELECT id,timestamp,DTU_Motor_RPM_897,DTU_throttle_position_897,BMS_pack_current_768,BMS_pack_vtg_768,DTU_dc_link_voltg_897,Vehicle_Speed_actual_921,DTU_Resultant_Current_896,DTU_Inverter_control_mode_516,DTU_q_axis_curr_901,DTU_d_axis_curr_901,Id_reference_922,Iq_reference_922,BMS_soc_percentage_774,BMS_sop_watt_774,rx_gear_position_920 FROM bytebeam.can_parsed_dbc_base_v0 WHERE (timestamp >= now() - interval 10 minute) ORDER BY id, timestamp")
arr_index=index.result_rows
index_df1 = pd.DataFrame(arr_index)
index_df1.columns=index.column_names
print(index_df1)

if index_df1.empty:
    formatted_time_utc = time.strftime("%Y-%m-%d %H:%M:%S")
    utc_time = datetime.strptime(formatted_time_utc, "%Y-%m-%d %H:%M:%S")
    local_time = utc_time.astimezone(tzlocal())
    print('No data found at Time: ' + str(local_time))

    id               timestamp DTU_Motor_RPM_897 DTU_throttle_position_897  \
0    2 2023-10-09 05:44:24.236              None                      None   
1    2 2023-10-09 05:44:25.237              None                      None   
2    2 2023-10-09 05:44:26.237              None                      None   
3    2 2023-10-09 05:44:27.240              None                      None   
4    2 2023-10-09 05:44:28.240              None                      None   
..  ..                     ...               ...                       ...   
554  2 2023-10-09 05:53:45.687              None                      None   
555  2 2023-10-09 05:53:46.688              None                      None   
556  2 2023-10-09 05:53:47.688              None                      None   
557  2 2023-10-09 05:53:48.689              None                      None   
558  2 2023-10-09 05:53:49.689              None                      None   

    BMS_pack_current_768 BMS_pack_vtg_768 DTU_dc_link_voltg_897

In [7]:
grouped = index_df1.groupby('id')
grouped
index_df = {} 
for group_name, group_data in grouped:
    df_name = f'df_{group_name}'
    index_df[df_name] = group_data
index_df = pd.concat(index_df.values(), ignore_index=True)
index_df

,id,timestamp,DTU_Motor_RPM_897,DTU_throttle_position_897,BMS_pack_current_768,BMS_pack_vtg_768,DTU_dc_link_voltg_897,Vehicle_Speed_actual_921,DTU_Resultant_Current_896,DTU_Inverter_control_mode_516,DTU_q_axis_curr_901,DTU_d_axis_curr_901,Id_reference_922,Iq_reference_922,BMS_soc_percentage_774,BMS_sop_watt_774,rx_gear_position_920
0,2,2023-10-09 05:44:24.236,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,2023-10-09 05:44:25.237,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2,2023-10-09 05:44:26.237,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2,2023-10-09 05:44:27.240,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2,2023-10-09 05:44:28.240,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,2,2023-10-09 05:53:45.687,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
555,2,2023-10-09 05:53:46.688,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
556,2,2023-10-09 05:53:47.688,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
557,2,2023-10-09 05:53:48.689,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [10]:
indices_to_replace = [1, 85, 325, 445, 550]  # Example indices to replace
replacement_values = [100, 85, 71, 95, 72]  # Corresponding replacement values

for index, replacement in zip(indices_to_replace, replacement_values):
    index_df.at[index, 'DTU_throttle_position_897'] = replacement

indices_to_replace = [1, 85, 325, 445, 550]  # Example indices to replace
replacement_values = [10000,8700,7044,7658]  # Corresponding replacement values

for index, replacement in zip(indices_to_replace, replacement_values):
    index_df.at[index, 'DTU_Motor_RPM_897'] = replacement

In [ ]:
# condition = (index_df1['DTU_throttle_position_897'] > 70) & (index_df1['DTU_Motor_RPM_897'] > 6000)

# # Apply the conditions to filter the rows
# filtered_rows = index_df1[condition]

# # Create another DataFrame to append the filtered rows
# Throttle_df = pd.DataFrame()

# # Append the filtered rows to 'another_df'
# Throttle_df = Throttle_df.append(filtered_rows)
# Throttle_df = pd.concat([Throttle_df, pd.DataFrame(Throttle_df)], ignore_index=True)

# Throttle_df

In [ ]:
# condition1 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 4)
# condition2 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 3)
# condition3 = (index_df['DTU_throttle_position_897'] > 80) & (index_df['rx_gear_position_920'] == 2)
# condition4 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 1)
# condition5 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 'neutral')
# condition6 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] < 100)
# condition7 = (index_df['DTU_throttle_position_897'] < 10) & (index_df['DTU_Motor_RPM_897'].between(5000, 7000))
# condition8 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] > 6000)
# final_condition = condition1 | condition2 | condition3 | condition4 | condition5 | condition6 | condition7 | condition8

# filtered_df = index_df[final_condition]
# print(filtered_df)


In [12]:
import pandas as pd

# Define conditions
condition1 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 4)
condition2 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 3)
condition3 = (index_df['DTU_throttle_position_897'] > 80) & (index_df['rx_gear_position_920'] == 2)
condition4 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 1)
condition5 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 'neutral')
condition6 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] < 100)
condition7 = (index_df['DTU_throttle_position_897'] < 10) & (index_df['DTU_Motor_RPM_897'].between(5000, 7000))
condition8 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] > 6000)

final_condition = condition1 | condition2 | condition3 | condition4 | condition5 | condition6 | condition7 | condition8

# Filter DataFrame based on final_condition
filtered_df = index_df[final_condition]

# Extract the list of columns used in conditions
used_columns = ['DTU_throttle_position_897', 'DTU_Motor_RPM_897']

# Add timestamp
filtered_df['timestamp'] = pd.Timestamp.now()

# Select only the columns used in conditions and timestamp
filtered_df = filtered_df[used_columns + ['timestamp']]

print(filtered_df)

    DTU_throttle_position_897 DTU_Motor_RPM_897                  timestamp
1                         100             10000 2023-10-09 11:27:21.477946
85                         85              8700 2023-10-09 11:27:21.477946
325                        71              7044 2023-10-09 11:27:21.477946
445                        95              7658 2023-10-09 11:27:21.477946


In [18]:
import pandas as pd
condition = []
# Define conditions
condition1 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 4)
condition2 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 3)
condition3 = (index_df['DTU_throttle_position_897'] > 80) & (index_df['rx_gear_position_920'] == 2)
condition4 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 1)
condition5 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 'neutral')
condition6 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] < 100)
condition7 = (index_df['DTU_throttle_position_897'] < 10) & (index_df['DTU_Motor_RPM_897'].between(5000, 7000))
condition8 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] > 6000)

final_condition = condition1 | condition2 | condition3 | condition4 | condition5 | condition6 | condition7 | condition8
if final_condition == condition1:
    final_condition 
    condition.append(final_condition)




# Filter DataFrame based on final_condition
filtered_df = index_df[final_condition]

# Select all columns where any condition is true
used_columns = index_df.columns[final_condition]

# Add timestamp
filtered_df['timestamp'] = pd.Timestamp.now()

# Select the columns used in conditions and timestamp
filtered_df = filtered_df[used_columns.append(pd.Index(['timestamp']))]

print(filtered_df)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
import pandas as pd

# Define conditions
condition1 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 4)
condition2 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 3)
condition3 = (index_df['DTU_throttle_position_897'] > 80) & (index_df['rx_gear_position_920'] == 2)
condition4 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 1)
condition5 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['rx_gear_position_920'] == 'neutral')
condition6 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] < 100)
condition7 = (index_df['DTU_throttle_position_897'] < 10) & (index_df['DTU_Motor_RPM_897'].between(5000, 7000))
condition8 = (index_df['DTU_throttle_position_897'] > 70) & (index_df['DTU_Motor_RPM_897'] > 6000)

final_condition = condition1 | condition2 | condition3 | condition4 | condition5 | condition6 | condition7 | condition8

# Initialize an empty list to store the selected columns
selected_columns = []

# Check each condition
if condition1.any() or condition2.any() or condition3.any() or condition4.any() or condition5.any():
    selected_columns.extend(['DTU_Motor_RPM_897', 'DTU_throttle_position_897'])

if condition6.any() or condition7.any() or condition8.any():
    selected_columns.extend(['DTU_Motor_RPM_897', 'DTU_throttle_position_897'])

# Remove duplicates
selected_columns = list(set(selected_columns))

# Extract the selected columns from index_df
result_df = index_df[selected_columns]

print(result_df)


    DTU_Motor_RPM_897 DTU_throttle_position_897
0                None                      None
1               10000                       100
2                None                      None
3                None                      None
4                None                      None
..                ...                       ...
554              None                      None
555              None                      None
556              None                      None
557              None                      None
558              None                      None

[559 rows x 2 columns]


In [16]:
# for i in range(len(filtered_df)):    
#     fault_start_timestamp = filtered_df.iloc[i]['timestamp']
#     desired_id = filtered_df.iloc[i]['id']
#     start_time = fault_start_timestamp - row_range
#     end_time = fault_start_timestamp + row_range
#     index_pt = clickhouse_client.query(f"SELECT id,timestamp,DTU_Motor_RPM_897,DTU_throttle_position_897,BMS_pack_current_768,BMS_pack_vtg_768,DTU_dc_link_voltg_897,Vehicle_Speed_actual_921,DTU_Resultant_Current_896,DTU_Inverter_control_mode_516,DTU_q_axis_curr_901,DTU_d_axis_curr_901,Id_reference_922,Iq_reference_922,BMS_soc_percentage_774,BMS_sop_watt_774 FROM bytebeam.can_parsed_dbc_base_v0 WHERE id = '{str(desired_id)}' AND timestamp >= toDateTime64('{str(start_time)}', 3, 'UTC') AND timestamp <= toDateTime64('{str(end_time)}', 3, 'UTC') ORDER BY id, timestamp")
#     arr_index_pt = index_pt.result_rows
#     index_df1_pt = pd.DataFrame(arr_index_pt)
#     index_df1_pt.columns = index_pt.column_names
#     Throttle_condition_table = Throttle_utilization_table.append(index_df1_pt, ignore_index=True)
# Throttle_condition_table